# A demonstration of PANDAS data frames used to investigate CAO points

Author: Jon Ishaque
Commenced: 29th September 2021
GMIT SID: G00398244

This notebook extracts CAO points from the CAO website for 2019, 2020 and 2021. It loads data into pandas dataframes and uses pandas and python to compare points from different years.

 Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
 
 
*** 

In [263]:

#Package to make http requests
#(Requests: HTTP for Humans™ — Requests 2.26.0 documentation, 2021)
import requests as rq 

# Dates and time package
import datetime as dt

#pandas to load data into dataframes and use functionality to manipulate data and analyse data
import pandas as pd

#import regex package for searching strings
#(re — Regular expression operations — Python 3.10.0 documentation, 2021)

import re 

#import csv, deals with commas when writing to file
import csv

#use urlib to retrieve url as file for 2019 and 2020
import urllib.request as urlrq 

***
## 1. Loading CAO data into pandas.

Loading data into the notebook and pandas dataframes requires 3 different methods for each of the years 2019, 2020 and 2021 because for each year the data is in a different format:

2021 html webpage format

2020 Microsoft Excel format

2019 pdf

The next state of the notebook will explain how each years CAO points are accessed from the CAO website, backed up locally in their original format and imported into a pandas dataframe for each year either directly or from a saved csv..

The dataframes for each year are then joined up (1.4).



*** 
### 1.1 2021 points

#http://www.cao.ie/index.php?page=points&p=2021

The 2021 CAO points are presented in a web page. 
There are two pages one for level 6 & 7 and one for level 8.
The web page are saved and files and loaded into a dataframe. 

The function get2021() 
reads a webpage  accessed using the request library.
Each line of the web page is read and tested for CAO course and points line
with a regex. There are helper functions atomise fields into characters which may indicate, random selections and portfolio/interview assessment for courses. There is also a function to determing the institution form the course code. Each line is then written to a csv file.

Both the level 6 & 7 and level 8 are then loaded into a single dataframe df2021

The page header from the server should decode as per: *Content-Type: text/html; charset=iso-8859-1*
However, one line uses \x96 which isn't defined in iso-8859-1. Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96. The character in question was had an Irish foda on a level 8 course

In [264]:
# Get the current date and time.
#Create a string var,*now*. this is use in file names of back up copies of CAO points.
now = dt.datetime.now()

# Format as a string.
#global as used in functions
global nowstr
nowstr = now.strftime('%Y%m%d_%H%M%S')



###### Compile the regular expression so it is not compiled at each interation of the loop reading the webpage

###### Explanation of the regualar expression (re — Regular expression operations — Python 3.10.1 documentation, 2021):
('&#91;A-Z]{2}&#91;0-9&#93;{3} (.*)(&#91;0-9&#93;{3}))
&#91;
&#91;A-Z&#93;{2}           Any two upper case aphanumeric

&#91;0-9&#93;{3}           Any three digits 0-9

'  '                       Two spaces

(.*)(&#91;0-9&#93;{3})     Any amount of text before 3 numeric characters




In [265]:
#set reg ex
re_courses = re.compile('[A-Z]{2}[0-9]{3} (.*)') 
#(re — Regular expression operations — Python 3.10.1 documentation, 2021)




In [266]:
#Function to get HEI name from course code. using a switcher dict as
#oppose to messy if/else block
#(How to Implement Switch Case Functions in Python? [2021] | upGrad blog, 2021)
def getHEI(cc):
    switcher = {'AC' : 'American College',
    'AD' : 'National College of Art and Design',
    'AL' : 'Athlone Institute of Technology',
    'AS' : 'St. Angela`s College',
    'CI' : 'Irish College of Humanities & Applied Sciences',
    'BY' : 'IBAT College Dublin',
    'CK' : 'University College Cork (NUI)',
    'CM' : 'Marino Institute of Education',
    'CR' : 'Cork Institute of Technology',
    'CT' : 'CCT College Dublin',
    'CW' : 'Institute of Technology Carlow',
    'DB' : 'Dublin Business School',
    'DC' : 'Dublin City University',
    'DK' : 'Dundalk Institute of Technology',
    'DL' : 'Dun Laoghaire Institute of Art Design and Technology',
    'DN' : 'University College Dublin (NUI)',
    'DS' : 'Dorset College',
    'GA' : 'Galway-Mayo Institute of Technology',
    'GB' : 'Galway Business School',
    'GC' : 'Griffith College',
    'GY' : 'National University of Ireland Galway',
    'ID' : 'ICD Business School',
    'LC' : 'Limerick Institute of Technology',
    'LM' : 'University of Limerick',
    'LY' : 'Letterkenny Institute of Technology',
    'MH' : 'Maynooth University',
    'MI' : 'Mary Immaculate College',
    'MU' : 'Pontifical University St Patricks College',
    'NC' : 'National College of Ireland (NCI)',
    'NM' : 'St Nicholas Montessori College Ireland',
    'PC' : 'Carlow College St. Patricks',
    'RC' : 'RCSI University of Medicine & Health Sciences',
    'SG' : 'Institute of Technology Sligo',
    'TL' : 'Institute of Technology Tralee',
    'TR' : 'Trinity College Dublin',
    'TU' : 'Technological University Dublin',
    'WD' : 'Waterford Institute of Technology'
    }
    cc = cc[:2]
    return  switcher.get(cc)


In [267]:
#helper function to AQA,* and # indicators from points values
def points_to_arr(s):
    AQA=''
    portfolio =''
    points=''
    random = ''
    #check 1st char for #
    #print(s)
    if s[0]=='#':
        portfolio='#'# add to var
    #check final char for  *
    if s[-1] == '*':
        random ='*'
    points=''    
    if s.find('AQA') ==-1: #not AQA
        #strip ~ and * from start and end of s
        for i in s:
            if i.isdigit():
                #concat points string
                points = points + i
    else:
        AQA ="AQA" #return AQA as separate val as it will be separate column
        #return
    return [points, portfolio, random,AQA]

The following block of code iterates through each line of the csv file 

This part of the note part of the note book will load the web page content. A loop will read each line of web page and determine if it's content is relevant and write content to a csv file.

In [347]:

#global csv path list so the programme
#will find the csv files after function
global csv_files
csv_files = []
def get2021(path): #A function to create csv files from both the L6&7 and L8 webpages.
    #Get the both level 8 and 6/7 web pages
    #getheaders and determine contenttype
    #loop through response text lines
    #get level
    #print (path)
    if path.find('L8') >= 0 :
        level = '8'
        resp = rq.get('http://www2.cao.ie/points/l8.php',
                      headers={"content-type":"text"})
    elif path.find('L67') >= 0 :
        level ='6/7'        
        resp = rq.get('http://www2.cao.ie/points/l76.php', 
                      headers={"content-type":"text"})
    else:
        level = ''        
    original_encoding = resp.encoding
    # Change to cp1252, which handles accented characters
    resp.encoding = 'cp1252'
    # Create a file path for the original data. 2021
    pathhtml = path + nowstr + '.html'
    # Save the original html file.
    with open(pathhtml, 'w') as f:
        f.write(resp.text)
    #set var to count lines for cross check with webpage
    no_lines = 0
    path = path+'.csv'
    #add csv name to array so as to access when loading into df
    csv_files.append(path)
    with open(path, 'w') as f:
        #write csv header
        '''
        PREFIXES:
        R1 = Round 1 points
        *     Not all on this points score were offered places.
        #     Test / Interview / Portfolio / Audition
        AQA   All qualified applicants
        v     New competition for available places        '''
        linesplit = ['Course Code','Course title','R1_21',
                     'Po1_21','Rn1_21','AQA1_21','R2_21',
                     'Po2_21','Rn2_21','AQA2_21',
                     'HEI','Level','Year']
        f.write(','.join(linesplit) + '\n')
        for line in resp.iter_lines():
            #problem with bytes
            #so convert str to bytes
            dline = line.decode('cp1252')
            #check if line mathces reg exp pattern. If so, do something.
            if re_courses.fullmatch(dline):
                no_lines +=1
                #get first five chars - course code
                course_code = dline[:5]
                #course title
                course_title = dline[7:57]
                #r1 points
                round_1 = dline[60:65].rstrip() # get five chars, remove white space
                #if round 1 not blank call fn points_to_arr
                if len(round_1) > 0:
                    round_1= points_to_arr(round_1)
                    #assign vals from returned array
                    pts1 = round_1[0]
                    plo1 = round_1[1]
                    rnd1 = round_1[2]
                    AQA1 = round_1[3]
                else: 
                    pts1 = ''
                    plo1 = ''
                    rnd1 = ''
                    AQA1 = ''
                #r2 points
                round_2 = dline[67:].rstrip() # get four chars, remove white space
                #if round 2 not blank call fn points_to_arr
                if len(round_2) > 0:
                    round_2= points_to_arr(round_2)
                    #assign vals from returned array
                    pts2 = round_2[0]
                    plo2 = round_2[1]
                    rnd2 = round_2[2]
                    AQA2 = round_2[3]
                else: 
                    pts2 = ''
                    plo2 = ''
                    rnd2 = ''
                    AQA2 = ''
                #print (course_code)
                #get the instituion name
                HEI =getHEI(course_code)
                #print (HEI)
                # create an array of the fields for the csv line
                linesplit = [course_code,course_title,pts1,plo1,rnd1,AQA2,pts2,plo2,rnd2,AQA2,HEI,level,'2021']
                #print (linesplit)
                #debug
                #print(f"'{course_code} {dline} r1: {round_1} r2: {round_2}'")
               # print((','.join(linesplit) + '\n'))
                # Rejoin the array values with commas in between. ie.comma separated
                f.write(','.join(linesplit) + '\n')
    print (f"{path}number of lines is", {no_lines}) #print file path and number of lines as confirmation
    path=''
#check this number is correct


In [348]:
# The file path for the csv file.
path_2021_L8 = 'data/cao2021_L8_' + nowstr 
path_2021L67 ='data/cao2021_L67_' + nowstr 

#call the get2021 function with paths to save the html file and create a csv file

In [349]:
get2021(path_2021_L8)
get2021(path_2021L67)

data/cao2021_L8_20211213_214528.csvnumber of lines is {949}
data/cao2021_L67_20211213_214528.csvnumber of lines is {416}


#### NB: 949 L8 courses on CAO website verified on 10th November 2021
#### 416 L6/7 courses on CAO website verified on 15th November 2021

Join L8 & L6/7 courses into one dataframe

In [352]:
# loop over the list of csv files
#(Pandas, 2021)

#print(csv_files)
df2021 = pd.DataFrame()
for f in csv_files:      
    # read the csv file with correc encoding
    #print(f)
    df_temp = pd.read_csv(f,encoding='cp1252') 
    #create a dataframe with all 2021 data
    df2021 = df2021.append(df_temp, ignore_index = True)

#debug
#print(df2021)
#df2021.to_csv('data/2021.csv')
    

*** 

### 1.2 2020 CAO points

http://www.cao.ie/index.php?page=points&p=2020 

The CAO points for 2020 stored in an excel(.xls) file. All level courses were indcluded in the one file
The excel file is downloaded using the urlrq package and backed up.
The file is read then downloaded into a  pandas data frame df2020.

Some unwanted columns are deleted and columns are renamed for consistency with other years.
*, AQA and # indicators are pulled out using pandas functionality and written to new columns.
 

In [353]:
# Create a file path for the original data.For backup
path2020 = 'data/cao2020_' + nowstr + '.xlsx'

#download to path
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',\
                  path2020)

('data/cao2020_20211213_214528.xlsx',
 <http.client.HTTPMessage at 0x22e2c4c36d0>)

Read the Excel file into a pandas dataframe 

In [362]:

#download and parse the excel spreadsheet
#skip first 10 header rows
#(pandas.read_excel — pandas 1.3.4 documentation, 2021)
df2020=pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',\
                 skiprows=10)
#print(df2020.iloc[123])

#check final row
#delete unwanted columns
df2020 = df2020.drop(['CATEGORY (i.e.ISCED description)','avp','v','Column1',\
              'Column2','Column3','Column4','Column5','Column6',\
              'Column7','Column8','Test/Interview #','EOS Random *'], 1)
#rename columns 
df2020 = df2020.rename(columns={'COURSE TITLE': 'Course title',\
                        'COURSE CODE2': 'Course Code',\
                        'R1 POINTS':'R1_20','R2 POINTS':'R2_20',\
                        'R1 Random *':'Rn1_20',\
                        'R2 Random*':'Rn2_20','LEVEL':'Level',\
                        'EOS':'EOS_20','EOS Mid-point':'Mid_20'})
#add some new columns so we will match our 2021 dataframe
df2020['Year'] =2020
df2020['Po1_20'] =''
df2020['Po2_20'] =''
df2020['AQA1_20'] =''
df2020['AQA2_20'] =''


#Pulling out AQA, # and  placing into own column.
#this dataset has either digits or #matric code for portfolio and AQA. Check for value,
#if it exists place in another column and set points col to blank.
#(Check For a Substring in a Pandas DataFrame Column, 2021)
df2020.loc[df2020['R1_20'].str.contains('#',na=False) ,'Po1_20'] = '#'  
df2020.loc[df2020['R2_20'].str.contains('#',na=False) ,'Po2_20'] = '#' 
df2020.loc[df2020['R1_20'].str.contains('AQA',na=False) , 'AQA1_20'] = 'AQA' 
df2020.loc[df2020['R2_20'].str.contains('AQA',na=False) , 'AQA2_20'] = 'AQA' 
#set to blank data which are either not needed or have been processed above
df2020.loc[df2020['EOS_20'].str.contains('matric',na=False) , 'EOS_20'] = '' 
df2020.loc[df2020['Mid_20'].str.contains('matric',na=False) , 'Mid_20'] = '' 
df2020.loc[df2020['R1_20'].str.contains('matric',na=False) , 'R1_20'] = '' 
df2020.loc[df2020['R1_20'].str.contains('AQA',na=False) , 'R1_20'] = ''
df2020.loc[df2020['R2_20'].str.contains('matric',na=False) , 'R2_20'] = '' 
df2020.loc[df2020['R2_20'].str.contains('AQA',na=False) , 'R2_20'] = '' 
#print(df2020.loc[df2020['Course Code']=='CR210']) # check we are picking up commas in csv fields  
#debug
#print(df2020)
#df2020.to_csv('data/2020.csv')

***

### 1.3 2019 CAO points
http://www2.cao.ie/points/lvl8_19.pdf


2019 CAO points are saved in a pdf file and tables within that file.
Like 2021 L8 and Levels 6&7 are stored on two web pages (pdfs)
For each pdf files: 
- the pdf is backed up
- the file is opened in camelot and the contents of each pdf table is appened to a dataframe.
- Columns are renamed and year column is added as and AQA, * and # are dealt with.
- A regfunction filters the dataframe to only hold courses.
- 

In [363]:
import camelot #use camelot package to extract tables from pdf files [7]

Read tables functions Parameters: url_path for path to read 2019 CAO points, csv_path to write to file


In [364]:
def read2019Tables(level):
    #download url to path
    # The file path for the url and csv/pdf saved file.
    url_path='http://www2.cao.ie/points/lvl'+level+'_19.pdf'
    csv_path = 'data/cao2019_L'+level+'_csv_' + nowstr + '.csv'
    pdf_path = 'data/cao2019_L'+level+'_' + nowstr + '.csv'
  
    #get the pdf and back it up
    urlrq.urlretrieve(url_path,pdf_path)
    #get the pdf tables o
    #(Camelot: PDF Table Extraction for Humans — Camelot 0.10.1 documentation, 2021)
    tables = camelot.read_pdf(url_path,\
                              pages='1-end',flavor='stream')
   
    #read all pages (Not able to read pdf tables spread across multiple pages · Issue #278 · atlanhq/camelot, 2021)
    #print(tables)
    #count tables and verify it matches the original file.
    tbl_cnt = len(tables)
    #export all tables - not what we really want
    #tables.export(path, f='csv', compress=False) 
    #tables[0]
    tables[1].parsing_report
    {
        'accuracy': 99.02,
        'whitespace': 12.24,
        'order': 1,
        'page': 10
    }
    
    i = 1 # exclude first header table 
    #iterate through the list of tables 
    data2019 = [] # empty list of tables
    for t in tables:    
        if i > 0: #exclude 1st table
            #write the table as a dataframe to listdata2019
            data2019.append(t.df) 
        i +=1 

    #combine all the dataframes in the list into one dataframe
    dfcombined = pd.concat(data2019)

    #add column headers
    dfcombined.columns = ['Course Code', 'Course title', 'EOS', 'Mid']
    #add year
    dfcombined['Year']= '2019'
    #add level
    if url_path.find('l8') >= 0 :
        
        dfcombined['Level'] = '8'
    elif url_path.find('l76') >= 0 :
        dfcombined['Level'] = '6_7'
    
    tbl_cnt
    #write to csv to store as back up.
    dfcombined.to_csv(csv_path)
    
    return dfcombined


In [365]:
#function calls to read 2019 pdf files from website
df2019L8=read2019Tables('76')
df2019L67=read2019Tables('8')

Filter df so only rows with course codes remain. (regex, Şirin and Siler, 2021)

In [366]:
#function to filter the dataframe on course code reg ex - i.e. get rid of institution title lines
def regex_filter(val): 
    regex= '[A-Z]{2}[0-9]{3}'
    if val:
        mo = re.search(regex,val)
        if mo:
            return True
        else:
            return False
    else:
        return False

In [367]:

dfs2019 = [df2019L67,df2019L8]#place both dfs into a list
df2019 = pd.concat(dfs2019,ignore_index=True) #concat dfs in list to make 1 df
df2019 = df2019[df2019['Course Code'].apply(regex_filter)] #filter the dataframe on reg ex above
#(df2019)
#df2019.to_csv('data/2019raw.csv')


###### reset index to remove indexes from appended dataframes.
reset because reindex will notwork with duplicate values indexes (Prasanna, 2021)


In [368]:
df2019 = df2019.reset_index(drop=True)
#Create and set year column
df2019['Year'] =2019
#create columns for potfolio, AQA and random - signal 1 even though there is only 1 for 2019
df2019['Rn1_19'] =''
df2019['Po1_19'] =''
df2019['AQA1_19'] =''



#deal with random, portfolio and AQAs. These occur only in in the EOS field so check this field
#for occurence, and strip out digits, replace digits and move flag to new column.
#This done by passing df rows to helper functions
#
#add HEI name #(How To Create a New Column Based on Values From Other Columns in Pandas, 2021)
def HEIrow(row):
    return getHEI(row['Course Code'])
df2019['HEI'] = ''

df2019['HEI'] = df2019.apply(lambda row: HEIrow(row), axis=1)

def getRandomCol(row):
    #treat this field as string
    if row['EOS'].find('*') > -1:
        return '*'
df2019['Rn1_19'] =  df2019.apply(lambda row: getRandomCol(row), axis=1)

def getPortFolCol(row):
    if row['EOS'].find('#') > -1:
        return '#'
df2019['Po1_19'] =  df2019.apply(lambda row: getPortFolCol(row), axis=1)
def getAQACol(row):
    if row['EOS'].find('AQA') > -1:
        return 'AQA'
df2019['AQA1_19'] =  df2019.apply(lambda row: getAQACol(row), axis=1)
#finally return digits if they exist to EOS
def getDigitsCol(row):
    points=''
    for i in row['EOS']:
            if i.isdigit():
                #concat points string
                points = points + i    
    return points
df2019['EOS'] =  df2019.apply(lambda row: getDigitsCol(row), axis=1)

In [369]:
#rename columns to homogenise with 2020 and 2021
df2019=df2019.rename(columns={'COURSE': 'Course title','Mid': 'Mid_19','EOS': 'EOS_19'})
#debug
#print(df2019)
#df2019.to_csv('data/2019.csv')

In [370]:
#display (df2019.loc[df2019['Course Code']=='CK791']) #AL861,CK201


In [371]:
#Get unique course list
#Create short dataframes
df2019_sh = df2019[['Course Code','Course title','Level']]
df2020_sh = df2020[['Course Code','Course title','Level']]
df2021_sh = df2021[['Course Code','Course title','Level']]
display(df2021_sh)

,Course Code,Course title,Level
0,AL801,Software Design for Virtual Reality and Gaming...,8
1,AL802,Software Design in Artificial Intelligence for...,8
2,AL803,Software Design for Mobile Apps and Connected ...,8
3,AL805,Computer Engineering for Network Infrastructur...,8
4,AL810,Quantity Surveying ...,8
...,...,...,...
1360,WD188,Applied Health Care ...,6/7
1361,WD205,Molecular Biology with Biopharmaceutical Scien...,6/7
1362,WD206,Electronic Engineering ...,6/7
1363,WD207,Mechanical Engineering ...,6/7


In [372]:
dfs=[df2021_sh,df2020_sh,df2021_sh]#(found, Hulsey and Hulsey, 2021)
all_sh_Courses=pd.concat(dfs,ignore_index=True)
#all_Courses[all_Courses.duplicated(subset=['Course code'])]
#clean date, remove duplicates on course code

all_sh_Courses.drop_duplicates(subset=['Course Code'],inplace=True,ignore_index=True)

all_sh_Courses

,Course Code,Course title,Level
0,AL801,Software Design for Virtual Reality and Gaming...,8
1,AL802,Software Design in Artificial Intelligence for...,8
2,AL803,Software Design for Mobile Apps and Connected ...,8
3,AL805,Computer Engineering for Network Infrastructur...,8
4,AL810,Quantity Surveying ...,8
...,...,...,...
1534,TU986,Print Media Technology and Management,8
1535,TU993,Early Childhood Care and Education,8
1536,TU994,Early Childhood Care and Education,8
1537,WD139,Civil Engineering,7


In [380]:
#debug
dfs=[df2021,df2020,df2021]#(found, Hulsey and Hulsey, 2021)
all_Courses=pd.concat(dfs,ignore_index=True)
#all_Courses[all_Courses.duplicated(subset=['Course code'])]
#clean date, remove duplicates on course code

#all_Courses.drop_duplicates(subset=['Course Code'],inplace=True,ignore_index=True)

all_Courses
#all_sh_Courses.to_csv('data/wholething.csv')
#display (all_Courses.loc[all_Courses['Course Code']=='AL801'])
#all_Courses.iloc[1538]

,Course title,R1_21,Po1_21,Rn1_21,AQA1_21,R2_21,Po2_21,Rn2_21,AQA2_21,HEI,...,R1_20,Rn1_20,R2_20,Rn2_20,EOS_20,Mid_20,Po1_20,Po2_20,AQA1_20,AQA2_20
0,Software Design for Virtual Reality and Gaming...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Athlone Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Software Design in Artificial Intelligence for...,313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Athlone Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Software Design for Mobile Apps and Connected ...,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Athlone Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Computer Engineering for Network Infrastructur...,321.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Athlone Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Quantity Surveying ...,328.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Athlone Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4189,Applied Health Care ...,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Waterford Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4190,Molecular Biology with Biopharmaceutical Scien...,NaN,NaN,NaN,NaN,262.0,NaN,NaN,NaN,Waterford Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4191,Electronic Engineering ...,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Waterford Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4192,Mechanical Engineering ...,172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Waterford Institute of Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.4 Join the data frames
<br>

In [375]:
#print(df2020)



In [376]:
#Set course code as index - default column to join frame on, makes the code much cleaner.
df2019.set_index('Course Code',inplace=True)
df2020.set_index('Course Code',inplace=True)
df2021.set_index('Course Code',inplace=True)
all_Courses.set_index('Course Code',inplace=True)

all_Courses=all_Courses.join(df2021[['R1_21','Rn1_21','Po1_21','AQA1_21','R2_21','Rn2_21','Po2_21','AQA2_21']])
all_Courses=all_Courses.join(df2020[['R1_20','Rn_1_20','Po_1_20','AQA1_20','R2_20','Rn2_20','Po2_20','AQA2_20','EOS_20','Mid_20' ]])
all_Courses=all_Courses.join(df2019[['Rn1_19','Po1_19','AQA1_19','EOS_19','Mid_19']])
all_Courses

KeyError: "None of ['Course Code'] are in the columns"

In [254]:
all_Courses_usefulCols=all_Courses[['Course Code','Course title' ,'Level','R1_21','R2_21','R1_20','R2_20','Mid_19','EOS_19','Mid_20','EOS_20']]

KeyError: "['Mid_19', 'EOS_19', 'Course Code'] not in index"

In [ ]:
all_Courses_usefulCols

In [207]:
#all_Courses_usefulCols['R2_21']=all_Courses_usefulCols['R2_21'].fillna(0)

all_Courses_usefulCols.to_csv('data/wholething.csv')


In [79]:

all_Courses_usefulCols.convert_dtypes(infer_objects=False).dtypes 
#all_Courses_usefulCols['R1_21'] = all_Courses_usefulCols['R1_21'].astype(int)
all_Courses_usefulCols.dtypes

NameError: name 'all_Courses_usefulCols' is not defined

---
### 2 Annual CAO Point Comparions Using Pandas.
<br>

In [ ]:
#Throw everything out by year, twist flip, stretch it!
#What are the categories? Year, HEI, L8 & L6/7 - should be able to infer 2020 levels for from 2019&2021  courses


In [ ]:
#One item of granularity that would enhance this analysis is places availabe on each course

---
## References:

Camelot-py.readthedocs.io. 2021. Camelot: PDF Table Extraction for Humans — Camelot 0.10.1 documentation. [online] Available at: <https://camelot-py.readthedocs.io/en/master/> [Accessed 23 November 2021].


 Hulsey, J. and Hulsey, J., 2021. How to iterate through a list of Data frames and drop all data if a specific string isnt found. [online] Stack Overflow. Available at: <https://stackoverflow.com/questions/55052989/how-to-iterate-through-a-list-of-data-frames-and-drop-all-data-if-a-specific-str> [Accessed 23 November 2021].

Medium. 2021. How To Create a New Column Based on Values From Other Columns in Pandas. [online] Available at: <https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d> [Accessed 23 November 2021].

Medium. 2021. Check For a Substring in a Pandas DataFrame Column. [online] Available at: <https://towardsdatascience.com/check-for-a-substring-in-a-pandas-dataframe-column-4b949f64852> [Accessed 23 November 2021].


upGrad blog. 2021. How to Implement Switch Case Functions in Python? [2021] | upGrad blog. [online] Available at: <https://www.upgrad.com/blog/how-to-implement-switch-case-functions-in-python/> [Accessed 23 November 2021].

GitHub. 2021. Not able to read pdf tables spread across multiple pages · Issue #278 · atlanhq/camelot. [online] Available at: <https://github.com/atlanhq/camelot/issues/278> [Accessed 23 November 2021].

 Pandas.pydata.org. 2021. pandas.DataFrame — pandas 1.3.4 documentation. [online] Available at: <https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html> [Accessed 23 November 2021].
 
Pandas.pydata.org. 2021. pandas.read_excel — pandas 1.3.4 documentation. [online] Available at: <https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html?highlight=read_excel#pandas.read_excel> [Accessed 23 November 2021].

 Pandas?, A., 2021. Appending to an empty DataFrame in Pandas?. [online] Stack Overflow. Available at: <https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas> [Accessed 23 November 2021].
 
 Docs.python.org. 2021. re — Regular expression operations — Python 3.10.0 documentation. [online] Available at: <https://docs.python.org/3/library/re.html> [Accessed 23 November 2021].
 
 Stack Overflow. Available at: <https://stackoverflow.com/questions/15325182/how-to-filter-rows-in-pandas-by-regex/48884429> [Accessed 23 November 2021].
 
 Docs.python-requests.org. 2021. Requests: HTTP for Humans™ — Requests 2.26.0 documentation. [online] Available at: <https://docs.python-requests.org/en/latest/index.html> [Accessed 23 November 2021].
 
 R. and Prasanna, V., 2021. Right way to reindex a dataframe?. [online] Stack Overflow. Available at: <https://stackoverflow.com/questions/68261366/right-way-to-reindex-a-dataframe> [Accessed 23 November 2021].











## End

upGrad blog. 2021. How to Implement Switch Case Functions in Python? [2021] | upGrad blog. [online] Available at: <https://www.upgrad.com/blog/how-to-implement-switch-case-functions-in-python/> [Accessed 23 November 2021].